---
title: "A look at different scenarios - Gains and Losses"
format:
  html:
    code-fold: true
    code-summary: "Show the code"
  docx: 
    fig-height: 11.6 in
---

In [14]:
import pandas as pd
from fagrants_module import fagrants
import seaborn as sns

def weighted_group_delta(grouped_df):
    return_dict = {}

    for year in range(2025, 2029):
        total_weight = grouped_df[f"ERP_{year}"].sum()

        return_dict[f"Population_{year}"] = total_weight

        weighted_mean = (
            grouped_df[f"Grant Delta_{year}"] * grouped_df[f"ERP_{year}"]
        ).sum() / total_weight

        return_dict[f"Grant Delta_{year}"] = weighted_mean

    return pd.Series(return_dict)

def grants_simulation(data, pc_min, pc_min_base):
    inst = fagrants.fagrants_model(
    data_path=data,
    per_capita_minimum=pc_min,
    per_capita_minimum_base=pc_min_base,
    )

    sim_grants = inst.run()

    sim_grants["Sim_i"] = sim_grants["UID"].str.split("_").str[1].astype(int)

    for year in range(2025, 2029):
        sim_grants[f"Grant Delta_{year}"] = (
            sim_grants[f"Grant_{year}"] - sim_grants[f"Grant_base_{year}"]
        )

    return sim_grants

def aggregate_delta(sim_grants):
    sim_grants_grouped = (
        sim_grants.groupby(["Jurisdiction", "ACLG", "Sim_i"])
        .apply(weighted_group_delta, include_groups=False)
        .reset_index()
    )

    sim_grants_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "Sim_i"],
        value_vars=[f"Grant Delta_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG ($)",
    )

    sim_grants_melted["Year"] = sim_grants_melted["Year"].str.split("_").str[1].astype(int)

    return sim_grants_melted, sim_grants_grouped

def per_capita_delta(sim_grants):
    sim_grants_grouped = (
    sim_grants.groupby(["ACLG", "Sim_i"])
    .apply(weighted_group_delta, include_groups=False)
    .reset_index()
    )

    for year in range(2025, 2029):
        sim_grants_grouped[f"Grant Delta Per Capita_{year}"] = sim_grants_grouped[f"Grant Delta_{year}"]/ sim_grants_grouped[f"Population_{year}"]

    sim_grants_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["ACLG", "Sim_i"],
        value_vars=[f"Grant Delta Per Capita_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG Per Capita($)",
    )

    sim_grants_melted["Year"] = sim_grants_melted["Year"].str.split("_").str[1].astype(int)

In [10]:
sim_grants_30 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.30, 0.30)

#sim_grants_20 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
#    0.20, 0.30)

#sim_grants_10 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
#    0.10, 0.30)

#sim_grants_0 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
#    0.00, 0.30)

In [13]:
sim_grants_30

,Jurisdiction,LGA,ACLG,UID,ERP_2024,Grant_2024,ERP_2025,Grant_base_2025,Grant_2025,ERP_2026,...,Grant_base_2027,Grant_2027,ERP_2028,Grant_base_2028,Grant_2028,Sim_i,Grant Delta_2025,Grant Delta_2026,Grant Delta_2027,Grant Delta_2028
0,New South Wales,The Council of the City of Sydney,UCC,TheCounciloftheCityofSydney_1,231086,5901858,238154.0,6.198876e+06,6.198876e+06,240869.0,...,6.663437e+06,6.663437e+06,246049.0,6.903611e+06,6.903611e+06,1,0.0,0.0,0.0,0.0
1,New South Wales,Hunter's Hill Council,UDS,Hunter'sHillCouncil_1,14036,441211,14465.0,4.499407e+05,4.499407e+05,14630.0,...,4.689512e+05,4.689512e+05,14945.0,4.845709e+05,4.845709e+05,1,0.0,0.0,0.0,0.0
2,New South Wales,Mosman Municipal Council,UDM,MosmanMunicipalCouncil_1,29071,908679,29960.0,9.687955e+05,9.687955e+05,30302.0,...,1.089323e+06,1.089323e+06,30953.0,1.123388e+06,1.123388e+06,1,0.0,0.0,0.0,0.0
3,New South Wales,Burwood Council,UDM,BurwoodCouncil_1,42498,1136510,43798.0,1.192356e+06,1.192356e+06,44297.0,...,1.276839e+06,1.276839e+06,45250.0,1.328299e+06,1.328299e+06,1,0.0,0.0,0.0,0.0
4,New South Wales,Lane Cove Municipal Council,UDM,LaneCoveMunicipalCouncil_1,41549,1123412,42820.0,1.179516e+06,1.179516e+06,43308.0,...,1.266079e+06,1.266079e+06,44239.0,1.316336e+06,1.316336e+06,1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537995,Northern Territory,Tiwi Islands Regional Council,RTM,TiwiIslandsRegionalCouncil_1000,2763,450412,2887.0,5.041882e+05,5.041882e+05,2910.0,...,4.588240e+05,4.588240e+05,2944.0,4.463492e+05,4.463492e+05,1000,0.0,0.0,0.0,0.0
537996,Northern Territory,Victoria Daly Regional Council,RTL,VictoriaDalyRegionalCouncil_1000,3314,589064,3463.0,5.898441e+05,5.898441e+05,3490.0,...,6.878901e+05,6.878901e+05,3531.0,7.797061e+05,7.797061e+05,1000,0.0,0.0,0.0,0.0
537997,Northern Territory,Wagait Shire Council,RTS,WagaitShireCouncil_1000,467,12407,488.0,1.270314e+04,1.270314e+04,492.0,...,1.350900e+04,1.350900e+04,498.0,1.397056e+04,1.397056e+04,1000,0.0,0.0,0.0,0.0
537998,Northern Territory,West Arnhem Regional Council,RTL,WestArnhemRegionalCouncil_1000,7449,1953690,7785.0,1.835507e+06,1.835507e+06,7845.0,...,2.097225e+06,2.097225e+06,7936.0,2.280103e+06,2.280103e+06,1000,0.0,0.0,0.0,0.0


In [15]:
sim_grants_30_melted_agg, sim_grants_30_grouped = aggregate_delta(sim_grants_30)

#sim_grants_30_melted_per_capita = per_capita_delta(sim_grants_30)

In [12]:
sim_grants_30_melted_agg

,Jurisdiction,ACLG,Sim_i,Year,Expected Change in GPG ($)
0,New South Wales,RAL,1,2025,0.0
1,New South Wales,RAL,2,2025,0.0
2,New South Wales,RAL,3,2025,0.0
3,New South Wales,RAL,4,2025,0.0
4,New South Wales,RAL,5,2025,0.0
...,...,...,...,...,...
395995,Western Australia,URS,996,2028,0.0
395996,Western Australia,URS,997,2028,0.0
395997,Western Australia,URS,998,2028,0.0
395998,Western Australia,URS,999,2028,0.0
